You can run this notebook directly on Google Colab

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/DaniAffCH/Vessel-Geometric-Transformers/blob/main/main.ipynb)

In [1]:
import sys

COLAB_RUNTIME = 'google.colab' in sys.modules
if COLAB_RUNTIME:
    !git clone https://github.com/DaniAffCH/Vessel-Geometric-Transformers.git
    !mv Vessel-Geometric-Transformers/* . 
    !pip install -q -r requirements.txt
else:
    !pip install -q -r requirements.txt
    !pre-commit autoupdate
    !pre-commit install



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip
[https://github.com/psf/black] already up to date!
[https://github.com/pycqa/isort] already up to date!
[https://github.com/PyCQA/flake8] already up to date!
[https://github.com/pre-commit/mirrors-mypy] already up to date!
pre-commit installed at .git/hooks/pre-commit


Loading the configuration

In [2]:
from src.utils import load_config
import os
from config import DatasetConfig, TrainerConfig, BaselineConfig, GatrConfig

config_path = os.path.join("config","config.yaml")

config = load_config(config_path)
dataset_config: DatasetConfig = config.dataset
trainer_config: TrainerConfig = config.trainer
baseline_config: BaselineConfig = config.baseline
gatr_config: GatrConfig = config.gatr

Loading the dataset

In [3]:
from src.data import VesselDataModule


data = VesselDataModule(dataset_config)
print(f'Train size: {len(data.train_set)}')
print(f'Validation size: {len(data.val_set)}')
print(f'Test size: {len(data.test_set)}')
print(data.train_set[2])

/home/daniaffch/Desktop/Uni/Deep_Learning/Vessel-Geometric-Transformers/.venv/lib/python3.10/site-packages/torch_geometric/data/dataset.py:238: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experim

Train size: 2999
Validation size: 599
Test size: 401
Data(pos=[8397, 3], wss=[8397, 3], pressure=[8397], face=[3, 16790], inlet_index=[214], label=Category.Single)


# TODO: analisi dati preliminare. Classi bilanciate? Statistiche sul dataset? Qualche plot figo.

# TODO: equivariance check! 

Testing

In [4]:
from src.lib import PointGeometricAlgebra, TranslationGeometricAlgebra, ScalarGeometricAlgebra, PlaneGeometricAlgebra

print(data.train_set[0].pos[0])
print(PointGeometricAlgebra.fromElement(data.train_set[0].pos[0].unsqueeze(0)))
print()

print(TranslationGeometricAlgebra.fromElement(data.train_set[1].wss)[:, :8])
print()

print(ScalarGeometricAlgebra.fromElement(data.train_set[0].pressure))
print()

print(PlaneGeometricAlgebra.fromElement(data.train_set[0].face.T))

tensor([ 0.0150, -0.1882,  0.0018])
tensor([[ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000,  0.0150, -0.1882,  0.0018,  1.0000,  0.0000]])

tensor([[1., 0., 0.,  ..., 0., 0., 0.],
        [1., 0., 0.,  ..., 0., 0., 0.],
        [1., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [1., 0., 0.,  ..., 0., 0., 0.],
        [1., 0., 0.,  ..., 0., 0., 0.],
        [1., 0., 0.,  ..., 0., 0., 0.]])

tensor([[133520.2500,      0.0000,      0.0000,  ...,      0.0000,
              0.0000,      0.0000],
        [133502.0156,      0.0000,      0.0000,  ...,      0.0000,
              0.0000,      0.0000],
        [133503.4844,      0.0000,      0.0000,  ...,      0.0000,
              0.0000,      0.0000],
        ...,
        [133278.6875,      0.0000,      0.0000,  ...,      0.0000,
              0.0000,      0.0000],
        [133279.0469,      0.0000,      0.0000,  ...,      0.0000,
              0.0000,      0.0000],
        [133595.

In [5]:
from src.trainer import VesselTrainer
from src.models import Gatr

trainer = VesselTrainer(trainer_config)


model = Gatr(gatr_config)
trainer.fit(model, data)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: danieleaffinita2000. Use `wandb login --relogin` to force relogin


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/home/daniaffch/Desktop/Uni/Deep_Learning/Vessel-Geometric-Transformers/.venv/lib/python3.10/site-packages/lightning/pytorch/loggers/wandb.py:396: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
/home/daniaffch/Desktop/Uni/Deep_Learning/Vessel-Geometric-Transformers/.venv/lib/python3.10/site-packages/lightning/pytorch/callbacks/model_checkpoint.py:652: Checkpoint directory /home/daniaffch/Uni/Deep_Learning/Vessel-Geometric-Transformers/ckpt exists and is not empty.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

   | Name             | Type              | Params | Mode 
----------------------------------------------------------------
0  | hsProjection     | EquiLinearLayer   | 172 K  | train
1  | backbone         | ModuleList        | 811

Sanity Checking DataLoader 0:   0%|          | 0/2 [00:00<?, ?it/s]

/home/daniaffch/Desktop/Uni/Deep_Learning/Vessel-Geometric-Transformers/.venv/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:424: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.
/home/daniaffch/Uni/Deep_Learning/Vessel-Geometric-Transformers/src/lib/geometricAlgebraOperations.py:85: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no lo

/home/daniaffch/Desktop/Uni/Deep_Learning/Vessel-Geometric-Transformers/.venv/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:424: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=15` in the `DataLoader` to improve performance.


...

Epoch 5: 100%|██████████| 750/750 [00:24<00:00, 30.73it/s, v_num=s7v9, val/loss=0.0346, val/acc=0.993, val/f1=0.993, train/loss=0.0329, train/acc=0.991, train/f1=0.991]

Epoch 5, global step 4500: 'val/loss' was not in top 1


Epoch 6: 100%|██████████| 750/750 [00:24<00:00, 30.72it/s, v_num=s7v9, val/loss=0.0413, val/acc=0.992, val/f1=0.991, train/loss=0.031, train/acc=0.994, train/f1=0.994] 

Monitored metric val/loss did not improve in the last 3 records. Best score: 0.031. Signaling Trainer to stop.
Epoch 6, global step 5250: 'val/loss' was not in top 1


Epoch 6: 100%|██████████| 750/750 [00:24<00:00, 30.68it/s, v_num=s7v9, val/loss=0.0413, val/acc=0.992, val/f1=0.991, train/loss=0.031, train/acc=0.994, train/f1=0.994]


Training Loop

In [6]:
from src.trainer import VesselTrainer
from src.models import BaselineTransformer

model = BaselineTransformer(baseline_config)

trainer.fit(model, data)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name           | Type               | Params | Mode 
--------------------------------------------------------------
0 | encoder        | TransformerEncoder | 2.2 K  | train
1 | embedder       | Linear             | 272    | train
2 | projection     | Linear             | 19.2 K | train
3 | loss_fn        | BCEWithLogitsLoss  | 0      | train
4 | train_accuracy | BinaryAccuracy     | 0      | train
5 | val_accuracy   | BinaryAccuracy     | 0      | train
6 | test_accuracy  | BinaryAccuracy     | 0      | train
7 | train_f1       | BinaryF1Score      | 0      | train
8 | val_f1         | BinaryF1Score      | 0      | train
9 | test_f1        | BinaryF1Score      | 0      | train
--------------------------------------------------------------
21.7 K    Trainable params
0         Non-trainable params
21.7 K    Total params
0.087     Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

In [8]:
trainer.test(model, data)

Restoring states from the checkpoint path at ckpt/last.ckpt


RuntimeError: Error(s) in loading state_dict for BaselineTransformer:
	Missing key(s) in state_dict: "encoder.layers.0.attention.q_proj.weight", "encoder.layers.0.attention.q_proj.bias", "encoder.layers.0.attention.k_proj.weight", "encoder.layers.0.attention.k_proj.bias", "encoder.layers.0.attention.v_proj.weight", "encoder.layers.0.attention.v_proj.bias", "encoder.layers.0.attention.out_proj.weight", "encoder.layers.0.attention.out_proj.bias", "encoder.layers.0.feedforward.0.weight", "encoder.layers.0.feedforward.0.bias", "encoder.layers.0.feedforward.2.weight", "encoder.layers.0.feedforward.2.bias", "encoder.layers.0.norm1.weight", "encoder.layers.0.norm1.bias", "encoder.layers.0.norm2.weight", "encoder.layers.0.norm2.bias", "encoder.layers.0.highway1.transform.weight", "encoder.layers.0.highway1.transform.bias", "encoder.layers.0.highway2.transform.weight", "encoder.layers.0.highway2.transform.bias", "embedder.weight", "embedder.bias", "projection.weight", "projection.bias". 
	Unexp

----